# Example: $p = 5$

## brute force tuple generator.py

In [ ]:
"""
description -- This program generates all indecomposable tuples for a given m and d value.
               The main method can be customized based on which m values we are interested in (i.e. m = p*q or m = p^2).
               The main method is currently written for m = p^2.
summary --
    1. The zmodmzstarset() function is called and the program generates this set for the given m value.
    2. The v_set() function is called and the program generates the V set for the given m and d value
       using the z_star set produced by zmodmzstar() function.
       This function calls other functions:
        - verify_not_all_pairs() that checks whether a tuple consists of all pairs. If this is true, the
          tuple is not an exceptional cycle and is not part of the exceptional cycles set. For efficiency, this tuple is
          not added to the V set either. This significantly reduces the number of tuples in the V set.
            * This means that there are fewer tuples that have to be checked with the verify_v_property() function, which
              requires a lot of mathematical computations.
        - verify_v_property() that checks if the tuple satisfies the B set property from our project summary document.
            * Our code treats the V and B set as the same set. (B set contains only ascending tuples from the V set).
    3. The e_set() function is called which finds all exceptional cycles in the V set. It separates the tuples into
       some pairs and no pairs.
    4. The indecomposable_set() function is called which finds all tuples in the no pairs set that are indecomposable.

"""

# IMPORTS
import math
import time
from itertools import combinations

# for folder organization
from pathlib import Path

def zmodmzstarset(m):
    """
    This function computes the z mod m z star set given an m value. This set contains integers ranging from [1,m).
    The integers in this set satisfy the following property: gcd(i, m) == 1.

    :param m: an odd integer which represents the degree in C_m: y^2=x^m-1
    :return: The set representing the z mod m z star set for the given m value.
    """
    z_star = []
    for i in range(1, m):
        if math.gcd(i, m) == 1:
            z_star.append(i)
    return z_star


def v_set(m, d, z_star):
    """
    This function computes the V set for a given m and d value. The function generates all d length "half tuples" whose
    values range from [1, (m-1)/2]. It then combines each half tuple with every other half tuple, forming a tuple of length
    2*d. It checks if the tuple satisfies the properties for the U set and then for the V set. If it does, the tuple is
    appended to the list.
    Only the tuples that satisfy the following conditions are added to the list:
    1. the elements of the tuple are ascending
    2. the sum of the elements = m*d
    3. the tuple is not composed entirely of pairs that sum to m
    4. verify_v_property() returns True

    :param m: an odd integer which represents the degree in C_m: y^2=x^m-1
    :param d: an integer whose range is [1, (m-1)/2]. This integer defines the length of the tuples produced.
          the tuples have length 2*d.
    :param z_star: a list representing the z mod m z star set for the given m value that was returned by the zmodmzstar() function.
    :return: a list represent the V set for the given m and d values.
    """
    count = 0
    small_tuple_list = []
    v_list = []
    print("These are the half tuple(s) in the U set: ")
    for combo in combinations(range(1, ((m-1)//2)+1), d):
        small_tuple_list.append(combo)
        print(combo)
        count += 1

    print("These are the tuple(s) in the V set: ")
    for half_tuple in small_tuple_list:
        i = 0
        while i < len(small_tuple_list):
            reverse_tuple = tuple(m-alpha for alpha in reversed(small_tuple_list[i]))
            if sum(half_tuple) + sum(reverse_tuple) == m*d:
                if is_not_all_pairs(half_tuple, reverse_tuple, m, d):
                    new_tuple = half_tuple + reverse_tuple
                    if verify_v_property(new_tuple, m, d, z_star):
                        print(new_tuple)
                        v_list.append(new_tuple)
            i += 1
    # print("These are the tuple(s) in the V set: ")
    # print(tuple_list)
    print("The number of tuple(s) is", len(v_list))
    print()
    return v_list


def is_not_all_pairs(tuple_1, tuple_2, m, d):
    """This function verifies whether a tuple is not composed entirely of pairs that sum to m."""
    i = 0
    j = d-1
    pair_count = 0
    while i < d and j > -1:
        if tuple_1[i] + tuple_2[j] == m:
            pair_count += 1
        i += 1
        j -= 1

    if pair_count == d:
        return False
    return True


def verify_v_property(u_tuple, m, d, z_star):
    """This function verifies whether a tuple meets the conditions necessary to be part of the V set."""
    t_count = 0
    for t in z_star:
        i = 0
        sum = 0
        while i < 2 * d:
            sum += ((u_tuple[i] * t) % m)
            i += 1
        if (sum / m) == d:
            t_count = t_count + 1
    if t_count == len(z_star):
        return True
    return False


def e_set(m, v_list):
    """
    This function computes the E set (set containing exceptional tuples). It creates the following lists:
    1. no_pairs -- represents the set with tuples that contain no pairs of elements that add to m.
    2. some_pairs -- represents the set with tuples that contain some but not all pairs of elements that add to m.

    :param m: an odd integer which represents the degree in C_m: y^2=x^m-1
    :param v_list: a list representing the V set for the given m and d values that was returned by the v_set() function.
    :return: tuple_list: a list representing the set of all tuples in the V set that are exceptional cycles, no_pairs: a list representing the subset of the exceptional cycles set that contains tuples containing no pairs
    """
    tuple_list = v_list[:]
    no_pairs = []
    some_pairs = []
    count = len(v_list)
    # This checks for all possible pair combinations in the tuple.
    for v_tuple in v_list:
        pair_count = 0
        for combo in combinations(v_tuple, 2):
            if sum(combo) == m:
                pair_count += 1
        if pair_count == int(len(v_tuple) / 2):
            count -= 1
            tuple_list.remove(v_tuple)
        if pair_count == 0:
            no_pairs.append(v_tuple)
        else:
            some_pairs.append(v_tuple)

    # PRINTING
    print("These are the tuple(s) in the E set: ")
    for e_tuple in tuple_list:
        print(e_tuple)
    print("The number of tuple(s) is", count)
    print("These tuples have no pairs that add to m =", m, )
    for no_pairs_tuple in no_pairs:
        print(no_pairs_tuple)
    print("The number of tuple(s) is", len(no_pairs))
    print("These tuples have some (but not all) pairs that add to m =", m, )
    for some_pairs_tuple in some_pairs:
        print(some_pairs_tuple)
    print("The number of tuple(s) is", len(some_pairs))
    print()
    return tuple_list, no_pairs


def indecomposable_set(m, d, no_pairs):
    """This functions determines which tuples in the no_pairs set are indecomposable and returns a list of those tuples."""
    tuple_list = no_pairs[:]
    for e_tuple in no_pairs:
        for i in range(2, d, 2):
            if e_tuple in no_pairs:
                for combo in combinations(e_tuple, i):
                    if sum(combo) % m == 0:
                        tuple_list.remove(e_tuple)
                        break
    # PRINTING
    print("These are the exceptional tuple(s) that are indecomposable: ")
    for ind_tuple in tuple_list:
        print(ind_tuple)
    print("The number of tuple(s) is", len(tuple_list))
    return tuple_list


def main():
    # Files
    base_dir = Path.cwd()
    output_dir = base_dir.parent/"outputs"/"brute_force_tuples_outputs"
    output_dir.mkdir(parents=True, exist_ok=True)

    # Create a list of primes. This will be useful to loop through.
    # For this example I will prepopulate list_of_primes to only contain 5.
    list_of_primes = [5]
    for p in list_of_primes:
        m = p**2
        for d in range(1, ((m-1)//2)+1):
            # d = (p+1)//2
            start_time = time.time()

            folder_name = f"m_{m}_output"
            folder_path = output_dir / folder_name
            folder_path.mkdir(parents=True, exist_ok=True)  # Create folder if it doesn't exist

            filename = f"m_{m}_d_{d}_output.txt"
            full_path = folder_path / filename


            z_star = zmodmzstarset(m)
            if full_path.exists():
                print(f"Skipping m = {m}, d = {d} — file already exists.")
                continue

            with open(full_path, "w") as file:
                file.write(f"For m = {m} and d = {d}\n")
                file.write(f"These are integers in the z mod m z star set for m = {m}\n {z_star}\n")
            v_tuple_list = v_set(m, d, z_star)
            e_tuple_list, no_pairs = e_set(m, v_tuple_list)
            indecomposable_list = indecomposable_set(m, d, no_pairs)
            total_time = time.time() - start_time

            # PRINTING
            with open(full_path, "a") as file:
                # PRINTING SUMMARY
                file.write(f"The program took {total_time} seconds to complete.\n")
                file.write(f"The number of tuple(s) in the V set is: {len(v_tuple_list)}\n")
                file.write(f"The number of tuple(s) in the E set is: {len(e_tuple_list)}\n")
                file.write(f"The number of tuple(s) with no pairs is: {len(no_pairs)}\n")
                file.write(f"The number of indecomposable tuple(s) is: {len(indecomposable_list)}\n\n")

                # PRINTING TUPLES
                # V set
                file.write(f"The tuples in the V set are:\n")
                for x in v_tuple_list:
                    file.write(f"{x}\n")
                file.write(f"The number of tuple(s) in the V set is: {len(v_tuple_list)}\n")
                file.write(f"\n")

                # E set
                file.write(f"The tuples in the E set are:\n")
                for x in e_tuple_list:
                    file.write(f"{x}\n")
                file.write(f"The number of tuple(s) in the E set is: {len(e_tuple_list)}\n")
                file.write(f"\n")

                # no pairs set
                file.write(f"The tuples in the no pairs set are:\n")
                for x in no_pairs:
                    file.write(f"{x}\n")
                file.write(f"The number of tuple(s) in the no pairs set is: {len(no_pairs)}\n")
                file.write(f"\n")

                # indecomposable set
                file.write(f"The indecomposable tuples are:\n")
                for x in indecomposable_list:
                    file.write(f"{x}\n")
                file.write(f"The number of indecomposable tuple(s) is: {len(indecomposable_list)}\n")
                file.write(f"\n")


main()



Skipping m = 25, d = 1 — file already exists.
Skipping m = 25, d = 2 — file already exists.
Skipping m = 25, d = 3 — file already exists.
Skipping m = 25, d = 4 — file already exists.
Skipping m = 25, d = 5 — file already exists.
Skipping m = 25, d = 6 — file already exists.
Skipping m = 25, d = 7 — file already exists.
Skipping m = 25, d = 8 — file already exists.
Skipping m = 25, d = 9 — file already exists.
Skipping m = 25, d = 10 — file already exists.
Skipping m = 25, d = 11 — file already exists.
Skipping m = 25, d = 12 — file already exists.


Since we have already generated the outputs for $p = 5$ via brute force, the outputs are stored in the output folder. This code is designed to skip any values of p such that the output already exists.

Let us take a look at the contents generated by this code for $p = 5, d = 3$.

In [ ]:
base_dir = Path.cwd()
full_path = base_dir.parent/"outputs"/"brute_force_tuples_outputs"/"m_25_output"/"m_25_d_3_output.txt"
if full_path.exists():
    with open(full_path, "r") as f:
        for line in f:
            print(line)
else:
    print(f"File was not found: {full_path}")

For m = 25 and d = 3

These are integers in the z mod m z star set for m = 25

 [1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 16, 17, 18, 19, 21, 22, 23, 24]

The number of tuple(s) in the U set is: 2971

The number of tuple(s) in the V/B set is: 224

The number of tuple(s) in the E set is: 4

The number of tuple(s) with no pairs is: 4

The number of indecomposable tuple(s) is: 4



The tuples in the V set are:

(1, 2, 3, 22, 23, 24)

(1, 2, 4, 21, 23, 24)

(1, 2, 5, 20, 23, 24)

(1, 2, 6, 19, 23, 24)

(1, 2, 7, 18, 23, 24)

(1, 2, 8, 17, 23, 24)

(1, 2, 9, 16, 23, 24)

(1, 2, 10, 15, 23, 24)

(1, 2, 11, 14, 23, 24)

(1, 2, 12, 13, 23, 24)

(1, 3, 4, 21, 22, 24)

(1, 3, 5, 20, 22, 24)

(1, 3, 6, 19, 22, 24)

(1, 3, 7, 18, 22, 24)

(1, 3, 8, 17, 22, 24)

(1, 3, 9, 16, 22, 24)

(1, 3, 10, 15, 22, 24)

(1, 3, 11, 14, 22, 24)

(1, 3, 12, 13, 22, 24)

(1, 4, 5, 20, 21, 24)

(1, 4, 6, 19, 21, 24)

(1, 4, 7, 18, 21, 24)

(1, 4, 8, 17, 21, 24)

(1, 4, 9, 16, 21, 24)

(1, 4, 10, 15, 21, 24)

(1, 4, 

The code separates the tuples into different sets. If desired, you may add print statements in the `main` function to print the contents of these sets. For simplicity, we have did not print the contents of every set.

## shioda tuples generator.py

In [ ]:
"""
description -- We noticed that for m = p^2 and d = (p+1)/2, the indecomposable tuples are the same tuples that are
               from Shioda's Lemma 5.5. This program will use the generalized equation to generate the necessary
               indecomposable tuples for m = p^2.
version -- This version generates the indecomposable tuples for all m values that satisfy m = p^2 for 2 < p < 1000.
           The tuples generated are in their original representation (i.e. not modified to so that elements i: i > m+1/2
           are written as i - m).
"""

# IMPORTS
import time
from pathlib import Path

def ind_tuple_generator(p, m):
    """ This function generates the Shioda tuples for a given p value.

    :param p: a prime number from the text file "primes.txt"
    :param m: the value p**2
    :return: a list containing all modified Shioda tuples which are also all the indecomposable tuples
    """
    ind_tuple_list = []
    for i in range(1, p):
        ind_tuple = tuple()
        coeff = 0
        while coeff <= p-1:
            ind_tuple = ind_tuple + (i + coeff*p,)
            coeff = coeff + 1
        ind_tuple = ind_tuple + (m - (p*i),)
        ind_tuple = tuple(sorted(ind_tuple))
        ind_tuple_list.append(ind_tuple)
    return ind_tuple_list

def main():
    # Files
    base_dir = Path.cwd()
    output_dir = base_dir.parent/"outputs"/"shioda_tuples_outputs"
    output_dir.mkdir(parents=True, exist_ok=True)

    # Create a list of primes
    # For this example I will prepopulate list_of_primes to only contain 5.
    list_of_primes = [5]

    # Loop through the list of primes. For each prime, generate all indecomposable tuples.
    for p in list_of_primes:
        m = p**2
        d = (p+1)//2
        start_time = time.time()
        filename = f"m_{m}_ind_output.txt"
        full_path = output_dir / filename

        if full_path.exists():
            print(f"Skipping m = {m}, d = {d} - file already exists.")
            continue

        with open(full_path, "w") as file:
            file.write(f"For m = {m} and d = {d}\n")

        ind_tuple_list = ind_tuple_generator(p, m)
        total_time = time.time() - start_time

        #PRINTING
        with open(full_path, "a") as file:
            # PRINTING SUMMARY
            file.write(f"The program took {total_time} seconds to complete. \n")
            file.write(f"The number of indecomposable tuple(s) is: {len(ind_tuple_list)}\n")

            # PRINTING TUPLES
            file.write("\nThe indecomposable tuples are:\n")
            for ind_tuple in ind_tuple_list:
                file.write(f"{ind_tuple}\n")



main()

Skipping m = 25, d = 3 - file already exists.


Here we have also already generated the indecomposable tuples for $p = 5$ and $d = 3$ using Shioda's equation. Let us take a look at these tuples.

In [ ]:
base_dir = Path.cwd()
full_path = base_dir.parent/"outputs"/"shioda_tuples_outputs"/"m_25_ind_output.txt"
if full_path.exists():
    with open(full_path, "r") as f:
        for line in f:
            print(line)
else:
    print(f"File was not found: {full_path}")

For m = 25 and d = 3

The program took 0.001001596450805664 seconds to complete. 

The number of indecomposable tuple(s) is: 4



The indecomposable tuples are:

(1, 6, 11, 16, 20, 21)

(2, 7, 12, 15, 17, 22)

(3, 8, 10, 13, 18, 23)

(4, 5, 9, 14, 19, 24)



## modified shioda tuples generator.py

In [ ]:
"""
description -- We noticed that for m = p^2 and d = (p+1)/2, the indecomposable tuples are the same tuples that are
               from Shioda's Lemma 5.5. This program will use the generalized equation to generate the necessary
               indecomposable tuples for m = p^2.
version -- This version generates the indecomposable tuples for all m values that satisfy m = p^2 for 2 < p < 1000.
           The tuples generated are modified so that elements i: i > m+1/2 are written as i - m.
"""

# IMPORTS
import time
from pathlib import Path

def ind_tuple_generator(p, m):
    """ This function generates the Shioda tuples for a given p value. The tuples are then modified so that any entry n that
    is greater than (m-1)/2 is rewritten as n-m.

        :param p: a prime number from the text file "primes.txt"
        :param m: the value p**2
        :return: a list containing all modified Shioda tuples which are also all the indecomposable tuples
    """
    ind_tuple_list = []
    for i in range(1, p):
        ind_tuple = tuple()
        coeff = 0
        while coeff <= p-1:
            ind_tuple = ind_tuple + (i + coeff*p,)
            coeff = coeff + 1
        ind_tuple = ind_tuple + (m - (p*i),)
        ind_tuple = tuple(sorted(ind_tuple))
        new_tuple = tuple()
        for n in ind_tuple:
            if n > (m-1)/2:
                n = n - m
            new_tuple = new_tuple + (n,)
        ind_tuple_list.append(new_tuple)
    return ind_tuple_list

def main():
    # Files
    base_dir = Path.cwd()
    output_dir = base_dir.parent/"outputs"/"modified_shioda_tuples_outputs"
    output_dir.mkdir(parents=True, exist_ok=True)

    # Create a list of primes
    # For this example I will prepopulate list_of_primes to only contain 5.
    list_of_primes = [5]

    # Loop through the list of primes. For each prime, generate all indecomposable tuples.
    for p in list_of_primes:
        m = p**2
        d = (p+1)//2
        start_time = time.time()
        filename = f"m_{m}_ind_f_output.txt"
        full_path = output_dir / filename

        if full_path.exists():
            print(f"Skipping m = {m}, d = {d} - file already exists.")
            continue

        with open(full_path, "w") as file:
            file.write(f"For m = {m} and d = {d}\n")

        ind_tuple_list = ind_tuple_generator(p, m)
        total_time = time.time() - start_time

        # PRINTING
        with open(full_path, "a") as file:
            # PRINTING SUMMARY
            file.write(f"The program took {total_time} seconds to complete. \n")
            file.write(f"The number of indecomposable tuple(s) is: {len(ind_tuple_list)}\n")

            # PRINTING TUPLES
            file.write("\nThe indecomposable tuples are:\n")
            for ind_tuple in ind_tuple_list:
                file.write(f"{ind_tuple} -> max value: {max(ind_tuple, key=abs)}\n")


main()

Skipping m = 25, d = 3 - file already exists.


Here we have also already generated the modified indecomposable tuples for $p = 5$ and $d = 3$ using Shioda's equation. Let us take a look at these tuples.

In [ ]:
base_dir = Path.cwd()
full_path = base_dir.parent/"outputs"/"modified_shioda_tuples_outputs"/"m_25_ind_f_output.txt"
if full_path.exists():
    with open(full_path, "r") as f:
        for line in f:
            print(line)
else:
    print(f"File was not found: {full_path}")

For m = 25 and d = 3

The program took 0.0009927749633789062 seconds to complete. 

The number of indecomposable tuple(s) is: 4



The indecomposable tuples are:

(1, 6, 11, -9, -5, -4) -> max value: 11

(2, 7, 12, -10, -8, -3) -> max value: 12

(3, 8, 10, -12, -7, -2) -> max value: -12

(4, 5, 9, -11, -6, -1) -> max value: -11



## max info generator.py

In [ ]:
"""
description -- This program generates the files in the "max_element_info_output" folder. It first generates the modified
               Shioda tuples (see folder labeled "modified_shioda_tuples_output"). It then identifies the element with the
               maximum absolute value and writes the rest of the elements in terms of this element. (This is used to
               generate the identity component matrices.)
"""

from pathlib import Path

def ind_tuple_generator(p, m):
    """ This function generates the Shioda tuples for a given p value. The tuples are then modified so that any entry n that
    is greater than (m-1)/2 is rewritten as n-m.

    :param p: a prime number from the text file "primes.txt"
    :param m: the value p**2
    :return: a list containing all modified Shioda tuples which are also all the indecomposable tuples
    """
    ind_tuple_list = []
    for i in range(1, p):
        ind_tuple = tuple()
        coeff = 0
        while coeff <= p-1:
            ind_tuple = ind_tuple + (i + coeff*p,)
            coeff = coeff + 1
        ind_tuple = ind_tuple + (m - (p*i),)
        ind_tuple = tuple(sorted(ind_tuple))
        new_tuple = tuple()
        for n in ind_tuple:
            if n > (m-1)/2:
                n = n - m
            new_tuple = new_tuple + (n,)
        ind_tuple_list.append(new_tuple)
    return ind_tuple_list

def main():
    # Files
    base_dir = Path.cwd()
    output_dir = base_dir.parent/"outputs"/"max_element_info_outputs"
    output_dir.mkdir(parents=True, exist_ok=True)

    # Create a list of primes
    # For this example I will prepopulate list_of_primes to only contain 5.
    list_of_primes = [5]

    # Loop through the list of primes. For each prime, generate all indecomposable tuples and max element info.
    for p in list_of_primes:
        m = p**2
        d = (p+1)//2
        filename = f"m_{m}_max_element_output.txt"
        full_path = output_dir / filename

        if full_path.exists():
            print(f"Skipping m = {m}, d = {d} - file already exists.")
            continue

        ind_tuple_list = ind_tuple_generator(p, m)

        with open(full_path, "a") as file:
            file.write(f"Maximum values for m = {m} and d = {d}:\n")
            for ind_tuple in ind_tuple_list:
                max_value = max(ind_tuple, key=abs)
                new_tuple = tuple()
                for n in ind_tuple:
                    if n != max_value:
                        new_tuple = new_tuple + (-1*n,)
                new_tuple = sorted(new_tuple, key=lambda x: abs(x))
                file.write(f"{max_value} : {new_tuple}\n")

main()


Skipping m = 25, d = 3 - file already exists.


Similar to our earlier example, this output file already exists. However, let us take a look at its contents.

In [ ]:
base_dir = Path.cwd()
full_path = base_dir.parent/"outputs"/"max_element_info_outputs"/"m_25_max_element_output.txt"
if full_path.exists():
    with open(full_path, "r") as f:
        for line in f:
            print(line)
else:
    print(f"File was not found: {full_path}")

Maximum values for m = 25 and d = 3:

11 : [-1, 4, 5, -6, 9]

12 : [-2, 3, -7, 8, 10]

-12 : [2, -3, 7, -8, -10]

-11 : [1, -4, -5, 6, -9]



## Summary

This is the overall logic of our Python scripts. We began our research by generating the tuples via brute force. However, after noticing that all indecomposable tuples for $m = p^2$ and $d = \frac{p+1}{2}$ are generated using Shioda's equation, we wrote a new script that was much more efficient for generating the tuples. We also modified these tuples to help generate the max element information for each tuple which we used to compute moment statistics.